In [10]:
import sympy as sp
import numpy as np
import pprint
import os
import os.path
import pickle
sp.init_printing()

In [11]:
def init(N):
    config = {
        91:((4,3),6),
        143:((4,4),2),

        437: ((5,5),3),

        493: ((5,5),3),

        59989:((8,8),3),

        376289:((10,10),3)
    }
    return config[N]

In [12]:
N = 91
dim, BLOCKSIZE = init(N)

In [13]:
def subs(term, sub_rule):
    return term if term ==1  or term == 0 else term.subs(sub_rule)

def split_2d_array(array):
    # append zeros so that array can be equally splitted

    if len(array[0])%BLOCKSIZE > 0:
      
        missing_columns = BLOCKSIZE-(len(array[0])%BLOCKSIZE)
    else:
        missing_columns = 0
    array = np.concatenate((np.array(array), np.zeros((array.shape[0],missing_columns))),axis=1 )

    split = len(array[0]) / BLOCKSIZE

    return np.split(array[::-1], indices_or_sections=split, axis=1)

In [16]:
q_dim, p_dim = dim


q = sp.symbols(' '.join([f'q{i}' for i in range(q_dim-2)])) #-2 because we dont consider the leading and trailing ones
p = sp.symbols(' '.join([f'p{i}' for i in range(p_dim-2)]))
p = [p]
p_total = [1] + list(p)+ [1]
# generate multiplication table
# pad with zeros to make all rows equal in length

rows = [[0]* i + [q_sym * p_sym for p_sym in p_total] + [0] * (len(q)-i) for i, q_sym in enumerate(q)]

# add first and last table row
rows = [list(p)+[1] + [0] * (len(q)+1)] + rows + [[0] * len(q) + p_total]



In [17]:
def get_block_overflow(blocksize, block_sum):
    overflow = len(bin(block_sum))-2 -  blocksize 
    return overflow if overflow > 0 else 0

def append_to_c_array(c_arr, number, subs_rule):
    print(number)
    new_cs = sp.symbols(' '.join(f'c{i}' for i in range(len(c_arr), len(c_arr)+number)))    
    #update subs rule
    new_cs = list(new_cs) if number > 1 else [new_cs]
    for c in new_cs:
        subs_rule[c] = 1

    c_arr += new_cs

In [18]:
ones_subs_rule = {sym:1 for sym in list(p) + list(q)}

ones_blocks= np.vectorize(subs)(np.array(rows), ones_subs_rule)

ones_blocks = split_2d_array(ones_blocks)


# int conversion necessary because otherwise zeros are 0.0
max_block_sums = [[''.join((row.astype(int).astype(str))) for row in block] for block in ones_blocks  ] # don't ask me about this ever
print(max_block_sums)

# convert to binary and sum up rows
# turn it around for binary conversion
max_block_sums = [sum(int(row[::-1],2) for row in block) for block in max_block_sums] 
[get_block_overflow(blocksize=BLOCKSIZE,block_sum=sum_) for sum_ in max_block_sums]
max_block_sums


[['001110', '011100', '111000', '110000']]


In [19]:
rows_with_carry = np.array(rows.copy())
blocks = split_2d_array(rows_with_carry)
c = []

N_bin = bin(N)

for i in range(len(blocks)):

    # update blocks with new carries,
    blocks = split_2d_array(rows_with_carry)

    ones_blocks= np.vectorize(subs)(np.array(blocks[i]), ones_subs_rule)
    
    # int conversion necessary because otherwise zeros are 0.0
    max_block_sum = [''.join((row.astype(int).astype(str))) for row in ones_blocks]

    # convert to binary and sum up rows,
    # turn it around for binary conversion,
    max_block_sum = sum(int(row[::-1],2) for row in max_block_sum)
    overflow_bits = get_block_overflow(blocksize=BLOCKSIZE, block_sum=max_block_sum)
    
    if overflow_bits:
        # catch overflow,
        carry_index_start = (i+1)*BLOCKSIZE
        carry_index_end = carry_index_start + overflow_bits 
         
        if carry_index_end >= len(N_bin)-4:
            carry_index_end = len(N_bin)-4
            overflow_bits = carry_index_end -carry_index_start

        if carry_index_end <= carry_index_start:
            continue

        column_diff = carry_index_end - len(rows_with_carry[0])
        if column_diff > 0:
            zeros = np.zeros((rows_with_carry.shape[0],column_diff))
            print(zeros.shape)
            print(zeros)
            print(rows_with_carry.shape)
            print(rows_with_carry)
            rows_with_carry = np.concatenate((rows_with_carry, zeros),axis=1)

        # append carries to initial matrix,
        carry_line = np.zeros(len(rows_with_carry[0]), dtype=object)
        print(f"overflow bits {overflow_bits}")
        append_to_c_array(c, overflow_bits, ones_subs_rule)
        carry_line[carry_index_start:carry_index_end] = c[-overflow_bits:]
        #immediately substract carries from last column of current block
        carry_line[(i+1)*BLOCKSIZE-1] = sp.Add(*[-carry_bit*2**(j+1) for j, carry_bit in enumerate(c[-overflow_bits:])])
        
        print(rows_with_carry)
        print(rows_with_carry.shape)
        #print(np.array([carry_line]).shape)
        rows_with_carry = np.concatenate((rows_with_carry,np.array([carry_line])))
        print(overflow_bits)
        print(carry_line) 
        pprint.pp(rows_with_carry)
print(N_bin)

# IMPROVEMENT: cut the carry for the 1 (last column in general)

0b1011011


In [20]:
# derive cost function
block = split_2d_array(rows_with_carry)

blocks


[array([[0, 0, 1, p0, 1, 0.0],
        [0, q1, p0*q1, q1, 0, 0.0],
        [q0, p0*q0, q0, 0, 0, 0.0],
        [p0, 1, 0, 0, 0, 0.0]], dtype=object)]

In [ ]:
N_bin_rev = N_bin[2:-1][::-1] # reverse N_bin to because blocks are ordered from LSB to MSB also discard LSB
# column wise addition, with 2 to the power of column position in block
cost_eq = [[sp.Add(*column) *2**i for i, column in enumerate(block.T)] for block in blocks ]
# add the columns of each block together
cost_eq = [sp.Add(*column) for column in cost_eq]

# substract N from   the constraint
pprint.pp(cost_eq)
cost_eq = [block_equation-int(N_bin_rev[block_inx*BLOCKSIZE:(block_inx+1) * BLOCKSIZE][::-1],2) for block_inx, block_equation in enumerate(cost_eq)]
cost_eq 

[2*p0*q0 + 5*p0 + 4*p1*q0 + 10*p1 + 9*q0 + 22]


In [ ]:
# pickle cost equation
dump_object = (p,q,tuple(c),cost_eq)
filepath = os.path.join(os.getcwd(), 'problems',f'{dim[0]}_{dim[1]}_{N}.pickle')
with open(filepath, 'wb') as f:
    pickle.dump(dump_object,f)

In [25]:
with open(filepath, 'rb') as f:
    print(pickle.load(f))

((p0, p1, p2, p3, p4, p5), (q0, q1, q2, q3, q4, q5), (c0, c1, c2, c3, c4, c5, c6, c7, c8, c9), [-8*c0 - 16*c1 + 2*p0*q0 + 4*p0*q1 + p0 + 4*p1*q0 + 2*p1 + 4*p2 + q0 + 2*q1 + 4*q2 - 2, c0 + 2*c1 - 8*c2 - 16*c3 - 32*c4 + p0*q2 + 2*p0*q3 + 4*p0*q4 + p1*q1 + 2*p1*q2 + 4*p1*q3 + p2*q0 + 2*p2*q1 + 4*p2*q2 + 2*p3*q0 + 4*p3*q1 + p3 + 4*p4*q0 + 2*p4 + 4*p5 + q3 + 2*q4 + 4*q5 - 5, c2 + 2*c3 + 4*c4 - 8*c5 - 16*c6 - 32*c7 + p0*q5 + 2*p0 + p1*q4 + 2*p1*q5 + 4*p1 + p2*q3 + 2*p2*q4 + 4*p2*q5 + p3*q2 + 2*p3*q3 + 4*p3*q4 + p4*q1 + 2*p4*q2 + 4*p4*q3 + p5*q0 + 2*p5*q1 + 4*p5*q2 + 2*q0 + 4*q1 - 2, c5 + 2*c6 + 4*c7 - 8*c8 - 16*c9 + p2 + p3*q5 + 2*p3 + p4*q4 + 2*p4*q5 + 4*p4 + p5*q3 + 2*p5*q4 + 4*p5*q5 + q2 + 2*q3 + 4*q4 - 2, c8 + 2*c9 + p5 + q5 - 5])
